## q-QLMS

In [ ]:
# If gspx is not installed, we add it to the path
import os, sys
gdir = os.path.dirname(os.getcwd())  # parent folder
sys.path.insert(0, gdir)

In [ ]:
import matplotlib.pyplot as plt
from gspx.utils.graph import make_sensor
from gspx.utils.display import plot_graph
from gspx.signals import QuaternionSignal
import numpy as np

from gspx.qgsp import QMatrix

nvertices = 20
ntaps = 8

In [ ]:
A1, coords = make_sensor(N=nvertices, seed=2)
Ai, _ = make_sensor(N=nvertices, seed=3)
Aj, _ = make_sensor(N=nvertices, seed=4)
Ak, _ = make_sensor(N=nvertices, seed=5)

A = QMatrix([A1, Ai, Aj, Ak])

In [ ]:
plot_graph(
    A.abs(), coords=coords,
    figsize=(8, 4), colormap='viridis',
    node_size=120)

In [ ]:
A.visualize(dpi=100)

In [ ]:
from gspx.qgsp import QGFT

qgft = QGFT()
qgft.fit(A)

plt.plot(np.real(qgft.eigc), np.imag(qgft.eigc), 'bo')
plt.show()

In [ ]:
### Are we confident that A @ V = V * lambda here?

# (A * qgft.Vq[:, 0])[:4]
# (qgft.Vq[:, 0] * eigq[0])[:4]

In [ ]:
X_shifted = A * qgft.Vq
diff = X_shifted - qgft.Vq

diff_norm_squared = (diff.transpose().conjugate() * diff).diag()
# This does not work! Quaternion mutiplication is not commutative and
# this order is the wrogn one.
# diff_norm_squared = (diff * diff.transpose().conjugate()).diag()
diff_norm_squared

In [ ]:
tv = np.sqrt(np.abs(diff_norm_squared).astype(float))

plt.scatter(np.real(qgft.eigc), np.imag(qgft.eigc), c=tv)
plt.colorbar()
plt.title("Total Variation of eigenvectors for each eigenvalue")
plt.xlabel("Real(eigvals)")
plt.ylabel("Imag(eigvals)")
plt.show()

In [ ]:
idx_freq = np.argsort(tv)

plt.figure()
plt.scatter(np.arange(len(idx_freq)), tv[idx_freq], c=tv[idx_freq])
plt.xlabel("Index of eigenvalues")
plt.ylabel("TV")
plt.title("Sorting eigenvalues by TV")
plt.colorbar()
plt.show()

In [ ]:
h_ideal = np.zeros(len(idx_freq))

# Bandwith of 20% the frequency support
bandwidth = int(len(idx_freq) / 5)
h_ideal[idx_freq[:bandwidth]] = 1

plt.figure()
plt.scatter(np.arange(len(idx_freq)), h_ideal[idx_freq], c=tv[idx_freq])
plt.xlabel("Index of eigenvalues")
plt.ylabel("Frequency response")

cbar = plt.colorbar()
cbar.set_label("TV of respective eigenvector", rotation=90)
plt.title("Ideal LPF")
plt.show()

In [ ]:
# Heat kernel in all 4 quaternion dimensions
k = 0.2
ss = np.exp(-k * np.arange(len(idx_freq)))

ss = QuaternionSignal.from_rectangular(
    np.hstack([ss[:, np.newaxis]] * 4)
)
ss.visualize()

In [ ]:
s = qgft.inverse_transform(ss)

obj = QuaternionSignal()
obj.samples = s.samples.ravel()
node_color = [tuple(rgba) for rgba in obj.to_rgba()]

plot_graph(
    A.abs(), coords=coords, colors=node_color,
    figsize=(8, 4), colormap='viridis',
    node_size=120)

In [ ]:
rnd = np.random.RandomState(seed=42)
err_amplitude = 0.2
nn = rnd.uniform(low=-err_amplitude, high=err_amplitude, size=len(ss))

sn = qgft.inverse_transform(ss + nn)
sn.samples = sn.samples.ravel()

In [ ]:
ss.visualize()

In [ ]:
(ss + nn).visualize()

In [ ]:
sn.visualize()

# Adaptive filtering with QLMS

In [ ]:
MAX_ITR = 1000
deg = 7

eig_vander = np.vander(qgft.eigc, N=deg, increasing=True)
eig_vander.shape

In [ ]:
idx_freq

In [ ]:

theta_ideal = h_ideal[:, np.newaxis]
print(eig_vander.shape, theta_ideal.shape)

learning_rates = [0.001, 0.01, 0.1, 0.3]

lms = LMS(max_iter=MAX_ITR, alpha=learning_rates, scale=True)
lms.fit(eig_vander, y=theta_ideal)